# Knex extraction

In [1]:
import knex

knex.init('ollama', 'llama3.1', 'http://localhost:11434') # Local LLM (direct)
# knex.init('ollama', 'llama3.1', 'http://127.0.0.1:5000') # Local LLM (through proxy)
# knex.init('openai', 'gpt-4o') # OPENAI llm

text = """
René Dubois naît le 17.8.1905 aux Verrières, et meurt le 16.11.1976 à Marin-Epagnier, protestant, de Buttes.
Fils d'Henri Adolphe, fonctionnaire postal.
Marié(e) à Suzanne Mina Arnoux, Française, fille de Jules Cyprien.
Licence en sciences économiques à Neuchâtel.
Officier instructeur dès 1929.
Chef d'état-major des troupes d'aviation et de DCA (1953-1955).
Commandant de la brigade légère 1, de la division 2 (1958-1961), du corps d'armée de campagne 1 (1962-1967).
""".strip()

# Extract the information from the text
information = knex.extraction(text)

# Analyze the extracted information
info_dfs = information.dataframes()
for key in info_dfs.keys():
    print(key)
    display(info_dfs[key])

persons


,name,gender,origins,religion,birth_date,birth_place,death_date,death_place,father_name,mother_name
0,René Dubois,None,French,Protestant,1905.08.17,Les Verrières,1976.11.16,Marin-Epagnier,Henri Adolphe Dubois,None
1,Henri Adolphe,None,None,None,None,None,None,None,None,None
2,Suzanne Mina Arnoux,female,French,None,None,None,None,None,Jules Cyprien,None
3,None,None,None,None,None,None,None,None,None,Jules Cyprien


relationships


,person1_name,person2_name,date_begin,date_end,relationship_type
0,René Dubois,Suzanne Mina Arnoux,None,None,marriage
1,None,René Dubois,None,None,None
2,Suzanne Mina Arnoux,René Dubois,None,None,None
3,None,René Dubois,None,None,marriage
4,None,Jules Cyprien,None,None,parent
5,Suzanne Mina Arnoux,Jules Cyprien,None,None,marriage


activities


,person_name,name,place,date_begin,date_end,activity_type,institution,discipline
0,René Dubois,Officier instructeur,None,1929,None,job,None,None
1,René Dubois,Chef d'état-major des troupes d'aviation et de...,None,1953,1955,job,None,None
2,René Dubois,Commandant de la brigade légère 1,None,1958,1961,job,None,None
3,René Dubois,Commandant du corps d'armée de campagne 1,None,1962,1967,job,None,None
4,René Dubois,Licence en sciences économiques à Neuchâtel,None,None,None,formation,None,None
5,None,fonctionnaire postal,None,None,None,None,None,None
6,None,None,None,None,None,None,None,None
7,None,None,None,None,None,None,postal service,None


# Knex knowledge

In [2]:
import knex

# Transform the extracted information into a graph
graph = knex.knowledge(information)

# Get the full generated graph in a global dataframe
graph_df = graph.to_dataframe()
graph_df.to_csv( './graph.csv', index=False)

# Visualize the generated graph
graph.get_visuals('./graph.html')

./graph.html


# Knex full pipeline

In [3]:
import knex

knex.init('ollama', 'llama3.1', 'http://localhost:11434') # Local LLM (direct)
# knex.init('ollama', 'llama3.1', 'http://127.0.0.1:5000') # Local LLM (through proxy)
# knex.init('openai', 'gpt-4o') # OPENAI llm

text = """
René Dubois naît le 17.8.1905 aux Verrières, et meurt le 16.11.1976 à Marin-Epagnier, protestant, de Buttes.
Fils d'Henri Adolphe, fonctionnaire postal.
Marié(e) à Suzanne Mina Arnoux, Française, fille de Jules Cyprien.
Licence en sciences économiques à Neuchâtel.
Officier instructeur dès 1929.
Chef d'état-major des troupes d'aviation et de DCA (1953-1955).
Commandant de la brigade légère 1, de la division 2 (1958-1961), du corps d'armée de campagne 1 (1962-1967).
""".strip()

# Execute the full pipeline
graph = knex.knowledge_extraction(text, verbose=True)

# Results
graph_df = graph.to_dataframe()
graph_df.to_csv('./graph.csv', index=False)

# Visualize the generated graph
graph.get_visuals('./graph.html')

Persons found in the text: René Dubois, Henri Adolphe, Suzanne Mina Arnoux, Jules Cyprien

==== Extracting information about: René Dubois ====
name: René Dubois
origins: French
religion: Protestant
birth_date: 1905.08.17
birth_place: Les Verrières
death_date: 1976.11.16
death_place: Marin-Epagnier
father_name: Henri Adolphe Dubois

==== Extracting information about: Henri Adolphe ====
name: Henri Adolphe

==== Extracting information about: Suzanne Mina Arnoux ====
name: Suzanne Mina Arnoux
gender: female
origins: French
father_name: Jules Cyprien

==== Extracting information about: Jules Cyprien ====
mother_name: Jules Cyprien

=== Extracting activities of: René Dubois ===
activities 0: 
    person_name: René Dubois
    name: Officier instructeur
    date_begin: 1929
    activity_type: job
activities 1: 
    person_name: René Dubois
    name: Chef d'état-major des troupes d'aviation et de DCA
    date_begin: 1953
    date_end: 1955
    activity_type: job
activities 2: 
    person_name: